### Ruchi Jha

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [2]:
data = pd.read_csv("employee.csv")

In [3]:
data.shape

(4277, 13)

In [4]:
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [5]:
data = data.drop(columns=['id', 'timestamp','country'])

In [6]:
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()

In [7]:
data = data.dropna()

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


In [9]:
data_train = data.copy()
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [10]:
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [11]:
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [12]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.shape

(4261, 25)

In [13]:
final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [14]:
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [15]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


In [16]:
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols

['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [17]:
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [18]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
2443,-1.255750,0,-0.494228,-0.626224,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2361,-0.756433,0,-0.494228,-0.626224,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3571,0.242199,0,-0.494228,1.800351,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
526,-1.255750,0,2.351152,1.800351,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1301,-1.755066,0,-2.201457,2.406995,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,-0.407631,0,-0.482532,-0.626224,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
346,0.242199,1,-0.494228,-0.019581,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2395,-1.255750,0,0.643924,-0.626224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
600,0.242199,1,-0.494228,-0.626224,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [19]:
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [20]:
reg.coef_

array([  191.86513059,    33.79374504,   193.27258007,   217.58050324,
          57.14375626,   -77.70028243,   223.02597278, -2184.65026975,
        1138.69739159,   839.78636738,   677.68185722,   536.54233933,
         999.22716271,   833.49675826,    54.1913111 ,   944.72802405,
         688.85523802,  3582.74276375,   371.12241118,   180.43699588,
         212.64970951,   168.13324324,  -270.2805768 ,  -181.02268049])

In [21]:
reg.intercept_

np.float64(6159.334036205276)

In [22]:
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

np.float64(205.3544377770245)

In [23]:
y_pred = reg.predict(X_test)

In [24]:
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)


Mean Squared Error: 11613.448726254579


Question 1.Preprocess Test data and get predictions

In [25]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [26]:
X_test

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
196,-1.723386,0,-0.464644,-0.626526,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2839,0.340412,1,-0.464644,2.396249,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
2425,1.888261,0,-0.464644,-0.626526,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1953,-0.691487,0,-0.464644,-0.626526,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
305,0.017616,1,-0.857785,-0.626526,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,0.856362,1,-0.464644,-0.626526,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3956,0.222155,0,0.001161,-0.626526,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2567,-0.175537,0,-0.464644,2.396249,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
3879,0.340412,1,-0.464644,-0.626526,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [27]:
y_hat = reg.predict(X_test)

In [28]:
y_hat

array([6689.58345138, 7770.75544529, 7090.28541533, 6697.46915732,
       6778.44207892, 7067.34846169, 6274.15715452, 6943.71246362,
       5883.02768566, 6691.08190223, 6977.51070735, 7073.22670601,
       6717.18880704, 8106.675504  , 6205.4551899 , 7450.59965135,
       6736.16350413, 7625.99670392, 6731.43241344, 6736.16350413,
       7838.96963856, 6386.33942333, 7174.00211514, 6340.19263088,
       6666.48346408, 7108.10430345, 7105.39778842, 6106.00674223,
       6615.43919514, 7173.76417471, 7744.21940303, 6725.16654033,
       6373.98637591, 7002.63734537, 6689.58345138, 7173.76417471,
       7471.7800122 , 7064.82058643, 6943.71246362, 5815.20330656,
       7720.02268013, 7467.88497124, 7327.32159258, 7682.833164  ,
       6793.30726039, 7224.40377977, 7919.153732  , 6987.26129795,
       6953.27960523, 7022.14469527, 7698.4170865 , 7181.55396211,
       7296.51173045, 6908.25308766, 6329.19566707, 7173.76417471,
       7238.64955629, 8275.33117546, 7482.51150026, 7073.22670

In [29]:
mae_test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Mean Abosulte Error of test data: ",mae_test)


Mean Abosulte Error of test data:  0.128491582291202


In [30]:
mse_test = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Mean Squared Error of test data: ",mse_test)


Mean Squared Error of test data:  223.65076717131944
